In [1]:
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [2]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_motif"

In [3]:
# Load nbS1-O1 circuit with connectivity matrix
circuit = obi.Circuit(name="nbS1-O1",
                      path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"),
                      matrix_path=os.path.join(cmat_root, "nbS1-O1", "connectivity_matrix.h5"))
display(circuit.connectivity_matrix.matrix)

<COOrdinate sparse matrix of dtype 'int64'
	with 87457567 stored elements and shape (211712, 211712)>

In [109]:
# Set up pair motif neuron sets

hex_nset = "nbS1-HEX0"
layers = ["2", "3", "4", "5", "6"]
inh_subtypes = ["hex_O1PV", "hex_O1Sst"]

# # (1) E->I sets per layer and INH subtype
# neuron1_filter = [{"node_set": hex_nset, "synapse_class": "EXC", "layer": _lay} for _lay in layers for _inh in inh_subtypes]
# neuron2_filter = [{"node_set": [hex_nset, _inh], "synapse_class": "INH", "layer": _lay} for _lay in layers for _inh in inh_subtypes]

# conn_ff_filter = {"nsyn": {"gt": 0}}
# conn_fb_filter = {"nsyn": 0}  # No feedback connection

# pair_selection = {"count": 1, "method": "max_nsyn_ff"}  # Selection based on max. number of synapses

# virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "POm") if (_lay == "6" and _inh == "hex_O1Sst") else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers for _inh in inh_subtypes]
# output_path = os.path.join(output_root, "E2I")

# # (2) I->E sets per layer and INH subtype
# neuron1_filter = [{"node_set": [hex_nset, _inh],  "synapse_class": "INH", "layer": _lay} for _lay in layers for _inh in inh_subtypes]
# neuron2_filter = [{"node_set": hex_nset, "synapse_class": "EXC", "layer": _lay} for _lay in layers for _inh in inh_subtypes]

# conn_ff_filter = {"nsyn": {"gt": 0}}
# conn_fb_filter = {"nsyn": 0}  # No feedback connection

# pair_selection = {"count": 1, "method": "max_nsyn_ff"}  # Selection based on max. number of synapses

# virtual_sources_to_ignore = ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", )
# output_path = os.path.join(output_root, "I2E")

# (3) E<->I sets per layer and INH subtype
neuron1_filter = [{"node_set": hex_nset, "synapse_class": "EXC", "layer": _lay} for _lay in layers for _inh in inh_subtypes]
neuron2_filter = [{"node_set": [hex_nset, _inh],  "synapse_class": "INH", "layer": _lay} for _lay in layers for _inh in inh_subtypes]

conn_ff_filter = {"nsyn": {"gt": 0}}
conn_fb_filter = {"nsyn": {"gt": 0}}  # Reciprocal connection

pair_selection = {"count": 1, "method": "max_nsyn_all"}  # Selection based on max. number of synapses

virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "POm") if (_lay == "6" and _inh == "hex_O1Sst") else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers for _inh in inh_subtypes]
output_path = os.path.join(output_root, "ErcI")

node_set_list_op = "intersect"
motif_neuron_sets = obi.PairMotifNeuronSet(neuron1_filter=neuron1_filter, neuron2_filter=neuron2_filter, conn_ff_filter=conn_ff_filter, conn_fb_filter=conn_fb_filter, pair_selection=pair_selection, node_set_list_op=node_set_list_op)

names = [f"{os.path.split(output_path)[-1]}-{_inh}-{hex_nset}-L{_lay}" for _lay in layers for _inh in inh_subtypes]

In [113]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    neuron_set=motif_neuron_sets,
)

coupled_scan = obi.CoupledScan(form=circuit_extractions_form, output_root=output_path, coordinate_directory_option="ZERO_INDEX")

In [114]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
coupled_scan.coordinate_instances(display=True)

[2025-06-17 11:23:45,728] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-17 11:23:45,730] INFO: initialize.virtual_sources_to_ignore: [('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical', 'POm')]
[2025-06-17 11:23:45,731] INFO: neuron_set.neuron1_filter: [{'node_set': 'nbS1-HEX0', 'synapse_class': 'EXC', 'layer': '2'}, {'node_set': 'nbS1-HEX0', 'synapse_class': 'EXC', 'layer': '2'}, {'node_set': '

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['initialize', 'virtual_sources_to_ignore'], value=('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',)), SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron1_filter'], value={'node_set': 'nbS1-HEX0', 'synapse_class': 'EXC', 'layer': '2'}), SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron2_filter'], value={'node_set': ['nbS1-HEX0', 'hex_O1PV'], 'synapse_class': 'INH', 'layer': '2'})], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-O1', path='/Users/pokorny/Data/Circuits/nbS1-O1/circuit_config_postfix2.json', matri

In [108]:
coupled_scan.execute(processing_method='run')

[2025-06-17 11:15:26,540] INFO: None
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (2)


Copying containerized .h5 morphologies: 100%|████████████████████████████████████████████████████| 2/2 [00:00<00:00, 420.17it/s]

Copied 2 morphologies into container (0 already existed)
Copying 2 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (2)
Copying mod files
Extraction DONE
[2025-06-17 11:18:11,477] INFO: create_bbp_workflow_campaign_config() not yet complete.


{0: None}

In [115]:
# Check circuits
import json
from bluepysnap import Circuit

c = circuit.sonata_circuit
nodes = c.nodes["S1nonbarrel_neurons"]
hex0_ids = nodes.ids("nbS1-HEX0")

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    with open(coupled_scan.output_root / str(_idx) / "id_mapping.json", "r") as f:
        id_map = json.load(f)
    parent_ids = id_map["S1nonbarrel_neurons"]["parent_id"]
    assert np.all(np.isin(parent_ids, hex0_ids))

    nodes = c.nodes['S1nonbarrel_neurons']
    edges = c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"]
    nids = np.hstack([nodes.ids({"synapse_class": "EXC"}), nodes.ids({"synapse_class": "INH"})])
    print(f"{names[_idx]:27s}>>> {nodes.size} neurons, {edges.size} (FF {len(edges.pathway_edges(source=nids[0], target=nids[1]))}, FB {len(edges.pathway_edges(source=nids[1], target=nids[0]))}) synapses, m-types: {nodes.get(nids, properties=['mtype']).to_numpy().flatten().tolist()}")



ErcI-hex_O1PV-nbS1-HEX0-L2 >>> 2 neurons, 83 (FF 21, FB 62) synapses, m-types: ['L2_IPC', 'L23_LBC']
ErcI-hex_O1Sst-nbS1-HEX0-L2>>> 2 neurons, 56 (FF 30, FB 26) synapses, m-types: ['L2_IPC', 'L23_MC']
ErcI-hex_O1PV-nbS1-HEX0-L3 >>> 2 neurons, 91 (FF 10, FB 81) synapses, m-types: ['L3_TPC:A', 'L23_NBC']
ErcI-hex_O1Sst-nbS1-HEX0-L3>>> 2 neurons, 102 (FF 11, FB 91) synapses, m-types: ['L3_TPC:A', 'L23_MC']
ErcI-hex_O1PV-nbS1-HEX0-L4 >>> 2 neurons, 84 (FF 16, FB 68) synapses, m-types: ['L4_SSC', 'L4_LBC']
ErcI-hex_O1Sst-nbS1-HEX0-L4>>> 2 neurons, 63 (FF 9, FB 54) synapses, m-types: ['L4_UPC', 'L4_MC']
ErcI-hex_O1PV-nbS1-HEX0-L5 >>> 2 neurons, 123 (FF 17, FB 106) synapses, m-types: ['L5_TPC:A', 'L5_NBC']
ErcI-hex_O1Sst-nbS1-HEX0-L5>>> 2 neurons, 81 (FF 23, FB 58) synapses, m-types: ['L5_TPC:A', 'L5_BTC']
ErcI-hex_O1PV-nbS1-HEX0-L6 >>> 2 neurons, 89 (FF 10, FB 79) synapses, m-types: ['L6_TPC:C', 'L6_NBC']
ErcI-hex_O1Sst-nbS1-HEX0-L6>>> 2 neurons, 73 (FF 21, FB 52) synapses, m-types: ['L6_BPC